In [1]:
import os, json, cv2, torch
import numpy as np
from pathlib import Path
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.font_manager as fm
# YOLOv5
from ultralytics import YOLO

# EfficientDet
from effdet import get_efficientdet_config, EfficientDet, DetBenchTrain, DetBenchPredict

# COCO API
try:
    from pycocotools.coco import COCO
    from pycocotools.cocoeval import COCOeval
    COCO_AVAILABLE = True
except ImportError:
    print("pycocotools 없음")
    COCO_AVAILABLE = False

In [2]:
# 경로 설정
BASE_DIR = Path.cwd().parent
IMG_DIR = BASE_DIR / "data/raw/images"
JSON_DIR = BASE_DIR / "data/raw/json_labels"
DATASET_YOLO = BASE_DIR / "processed/preprocessed_data/yolov5"
DATASET_EFFDET = BASE_DIR / "processed/preprocessed_data/efficientdet"
RESULT_DIR = BASE_DIR / "processed/results_comparison"
YOLO_WEIGHTS_FILE = RESULT_DIR / "yolov5su.pt"

for d in [DATASET_YOLO, DATASET_EFFDET, RESULT_DIR]:
    d.mkdir(parents=True, exist_ok=True)

In [3]:
# 한국어 폰트
def setup_korean_font():
    try:
        if os.name == 'nt':
            font_path = 'C:/Windows/Fonts/malgun.ttf'
            if os.path.exists(font_path):
                font_name = fm.FontProperties(fname=font_path).get_name()
                plt.rc('font', family=font_name)
            else:
                plt.rc('font', family='DejaVu Sans')
        elif os.name == 'posix':
            plt.rc('font', family='AppleGothic')
        
        plt.rcParams['axes.unicode_minus'] = False
        print("한글 폰트 설정 완료")
    except Exception as e:
        print(f"폰트 설정 실패: {e}")
        plt.rc('font', family='DejaVu Sans')

setup_korean_font()

한글 폰트 설정 완료


In [4]:
# 데이터 전처리
def preprocess_data():
    jsons = list(JSON_DIR.glob("*.json"))
    if not jsons:
        return {}, []
    
    train, temp = train_test_split(jsons, train_size=0.8, random_state=42)
    val, test = train_test_split(temp, train_size=0.5, random_state=42)
    splits = {'train': [], 'val': [], 'test': []}
    classes, class_to_idx = [], {}
    
    for split, files in zip(['train', 'val', 'test'], [train, val, test]):
        for j in tqdm(files, desc=f"Loading {split}"):
            with open(j, 'r', encoding='utf-8') as f:
                d = json.load(f)
            
            # 이미지 찾기
            img_path = None
            for ext in ['.jpg', '.png', '.jpeg', '.JPG', '.PNG', '.JPEG']:
                p = IMG_DIR / f"{j.stem}{ext}"
                if p.exists():
                    img_path = str(p)
                    break
            if not img_path:
                continue
            
            # 클래스
            name = f"{d['cate1']}_{d['cate3']}"
            if name not in classes:
                class_to_idx[name] = len(classes)
                classes.append(name)
            
            bbox = d['bndbox']
            splits[split].append({
                'image': img_path,
                'bbox': [bbox['xmin'], bbox['ymin'], bbox['xmax'], bbox['ymax']],
                'label': class_to_idx[name],
                'json_stem': j.stem
            })
    
    print(f"Dataset: train={len(splits['train'])}, val={len(splits['val'])}, test={len(splits['test'])}")
    return splits, classes

In [5]:
# YOLO datasets
def prepare_yolo_dataset(splits, classes):
    import yaml
    
    for split in ['train', 'val', 'test']:
        (DATASET_YOLO / 'images' / split).mkdir(parents=True, exist_ok=True)
        (DATASET_YOLO / 'labels' / split).mkdir(parents=True, exist_ok=True)
    
    for split, items in splits.items():
        for item in tqdm(items, desc=f"YOLO {split}"):
            with open(item['image'], 'rb') as f:
                img = cv2.imdecode(np.frombuffer(f.read(), np.uint8), cv2.IMREAD_COLOR)
            h, w = img.shape[:2]
            
            img_save = DATASET_YOLO / 'images' / split / f"{item['json_stem']}.jpg"
            cv2.imwrite(str(img_save), img)
            
            bbox = item['bbox']
            x_center = (bbox[0] + bbox[2]) / 2 / w
            y_center = (bbox[1] + bbox[3]) / 2 / h
            width = (bbox[2] - bbox[0]) / w
            height = (bbox[3] - bbox[1]) / h
            
            label_save = DATASET_YOLO / 'labels' / split / f"{item['json_stem']}.txt"
            with open(label_save, 'w') as f:
                f.write(f"{item['label']} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")
    
    with open(DATASET_YOLO / 'data.yaml', 'w', encoding='utf-8') as f:
        yaml.dump({
            'path': str(DATASET_YOLO),
            'train': 'images/train',
            'val': 'images/val',
            'test': 'images/test',
            'nc': len(classes),
            'names': classes
        }, f, allow_unicode=True)
    
    print(f"YOLO 데이터셋 준비 완료: {DATASET_YOLO}")

In [6]:
# EfficientDet 데이터셋
class EffDetDataset(Dataset):
    def __init__(self, data, img_size=512):
        self.data = data
        self.img_size = img_size
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        
        with open(item['image'], 'rb') as f:
            img = cv2.imdecode(np.frombuffer(f.read(), np.uint8), cv2.IMREAD_COLOR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        h, w = img.shape[:2]
        
        img = cv2.resize(img, (self.img_size, self.img_size))
        img_tensor = torch.from_numpy(img).permute(2, 0, 1).float() / 255.0
        
        bbox = item['bbox']
        bbox_scaled = [
            bbox[0] * self.img_size / w,
            bbox[1] * self.img_size / h,
            bbox[2] * self.img_size / w,
            bbox[3] * self.img_size / h
        ]
        
        return img_tensor, {
            'bbox': torch.tensor([bbox_scaled], dtype=torch.float32),
            'cls': torch.tensor([item['label']], dtype=torch.long),
            'img_scale': torch.tensor([1.0], dtype=torch.float32),
            'img_size': torch.tensor([self.img_size, self.img_size], dtype=torch.long)
        }

def collate_fn(batch):
    images = torch.stack([x[0] for x in batch])
    max_boxes = max([x[1]['bbox'].shape[0] for x in batch])
    
    bboxes, classes, scales, sizes = [], [], [], []
    for x in batch:
        bbox, cls = x[1]['bbox'], x[1]['cls']
        n = bbox.shape[0]
        if n < max_boxes:
            bbox = torch.cat([bbox, torch.zeros((max_boxes - n, 4))])
            cls = torch.cat([cls, torch.ones(max_boxes - n, dtype=torch.long) * -1])
        bboxes.append(bbox)
        classes.append(cls)
        scales.append(x[1]['img_scale'])
        sizes.append(x[1]['img_size'])
    
    return images, {
        'bbox': torch.stack(bboxes),
        'cls': torch.stack(classes),
        'img_scale': torch.stack(scales),
        'img_size': torch.stack(sizes)
    }

In [7]:
# YOLOv5 모델
def train_yolo(data_yaml, epochs=100):
    print("\n YOLOv5 학습 시작")
    model = YOLO(str(YOLO_WEIGHTS_FILE))
    
    results = model.train(
        data=str(data_yaml),
        epochs=epochs,
        imgsz=640,
        batch=16,
        name='yolov5_freshness',
        device='0' if torch.cuda.is_available() else 'cpu',
        patience=30,
        workers=0,
        project=str(RESULT_DIR)
    )
    
    print(f" YOLOv5 학습 완료")
    return model

def test_yolo(model, data_yaml):
    print("\n YOLOv5 테스트셋 평가 시작")
    
    test_metrics = model.val(
        data=str(data_yaml),
        split='test',
        project=str(RESULT_DIR),
        name='yolov5_test'
    )
    
    yolo_test_metrics = {
        'mAP50': float(test_metrics.box.map50),
        'mAP50_95': float(test_metrics.box.map),
        'precision': float(test_metrics.box.mp),
        'recall': float(test_metrics.box.mr)
    }
    
    print(f"YOLOv5 테스트 완료")
    print(f"  mAP@0.5: {yolo_test_metrics['mAP50']:.3f}")
    print(f"  mAP@0.5:0.95: {yolo_test_metrics['mAP50_95']:.3f}")
    print(f"  Precision: {yolo_test_metrics['precision']:.3f}")
    print(f"  Recall: {yolo_test_metrics['recall']:.3f}")
    
    # YOLO metrics 저장
    yolo_metrics_path = RESULT_DIR / 'yolo_metrics.json'
    with open(yolo_metrics_path, 'w', encoding='utf-8') as f:
        json.dump({
            'summary': yolo_test_metrics
        }, f, indent=4, ensure_ascii=False)
    print(f"YOLO metrics 저장: {yolo_metrics_path}")
    
    return yolo_test_metrics

In [8]:
# COCO 형태변환
def create_coco_annotations(splits, classes):    
    coco_categories = []
    for idx, name in enumerate(classes): 
        coco_categories.append({
            "id": idx, 
            "name": name, 
            "supercategory": "freshness"
        })

    for split in ['train', 'val', 'test']:
        coco_images = []
        coco_annotations = []
        
        coco_data = {
            'info': {
                "description": f"Custom Dataset - {split} Set",
                "version": "1.0",
                "year": 2025,
                "contributor": "Contributor",
                "date_created": "2025/11/11"
            },
            'licenses': [{"id": 0, "name": "Unknown", "url": ""}],
            'categories': coco_categories,
            'images': coco_images,
            'annotations': coco_annotations
        }
        
        ann_id = 0
        
        for img_id, item in enumerate(splits[split]):
            try:
                with open(str(item['image']), 'rb') as f:
                    img = cv2.imdecode(np.frombuffer(f.read(), np.uint8), cv2.IMREAD_COLOR)
                h, w = img.shape[:2]
            except Exception as e:
                print(f"이미지 로드 실패 {item['image']}: {e}")
                continue
            
            coco_data['images'].append({
                'id': img_id,
                'file_name': str(item['image']),
                'width': w,
                'height': h
            })
            
            bbox = item['bbox']
            coco_data['annotations'].append({
                'id': ann_id,
                'image_id': img_id,
                'category_id': item['label'],
                'bbox': [bbox[0], bbox[1], bbox[2] - bbox[0], bbox[3] - bbox[1]],
                'area': (bbox[2] - bbox[0]) * (bbox[3] - bbox[1]),
                'iscrowd': 0
            })
            ann_id += 1
        
        anno_path = DATASET_EFFDET / f'coco_{split}.json'
        with open(anno_path, 'w') as f:
            json.dump(coco_data, f, indent=4)
        
        print(f"COCO {split} annotations: {anno_path}")
    
    return DATASET_EFFDET / 'coco_test.json'

In [9]:
# EfficientDet 모델
def train_efficientdet(splits, classes, epochs=100):
    print("\n EfficientDet 학습 시작")
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    num_classes = len(classes)
    
    gt_anno_file = create_coco_annotations(splits, classes)

    train_loader = DataLoader(EffDetDataset(splits['train']), batch_size=4,
                             shuffle=True, collate_fn=collate_fn, num_workers=0)
    val_loader = DataLoader(EffDetDataset(splits['val']), batch_size=4,
                           collate_fn=collate_fn, num_workers=0)
    
    config = get_efficientdet_config('tf_efficientdet_d0')
    config.num_classes = num_classes
    config.image_size = (512, 512)
    
    model = DetBenchTrain(EfficientDet(config, pretrained_backbone=True), config)
    model.to(device)
    
    optimizer = torch.optim.AdamW(model.parameters(), lr=0.01, weight_decay=0.0001)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
    
    best_loss = float('inf')
    patience_counter = 0
    max_patience = 30
    
    train_losses, val_losses = [], []

    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for images, targets in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            images = images.to(device)
            targets = {k: v.to(device) for k, v in targets.items()}
            
            optimizer.zero_grad()
            output = model(images, targets)
            loss = output['loss'] if isinstance(output, dict) else output
            
            if torch.isnan(loss):
                print("NaN loss detected")
                continue
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 10.0)
            optimizer.step()
            train_loss += loss.item()
        
        train_loss /= len(train_loader)
        train_losses.append(train_loss)
        
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for images, targets in val_loader:
                images = images.to(device)
                targets = {k: v.to(device) for k, v in targets.items()}
                output = model(images, targets)
                loss = output['loss'] if isinstance(output, dict) else output
                val_loss += loss.item()
        
        val_loss /= len(val_loader)
        val_losses.append(val_loss) 
        print(f"Epoch {epoch+1}: Train={train_loss:.4f}, Val={val_loss:.4f}")

        if epoch == 0 or val_loss < best_loss:
            if val_loss < best_loss:
                best_loss = val_loss
            
            patience_counter = 0
            torch.save({
                'model_state_dict': model.model.state_dict(),
                'config': config
            }, RESULT_DIR / 'efficientdet_best.pth')
            print(f"Saved (Loss: {best_loss:.4f})")
        else:
            patience_counter += 1
            if patience_counter >= max_patience:
                print(f"Early stopping at epoch {epoch+1}")
                break
        
        scheduler.step()
    
    print(f"EfficientDet 학습 완료 (Best Loss: {best_loss:.4f})")

    plt.figure(figsize=(8, 5))
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.title("EfficientDet Training Loss Curve")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(RESULT_DIR / "efficientdet_loss_curve.png")
    plt.close()
    print(f" EfficientDet 학습 곡선 저장됨: {RESULT_DIR / 'efficientdet_loss_curve.png'}")

    return model, config

In [ ]:
def evaluate_efficientdet_with_confusion_matrix(config, splits, classes, device):
    print("\n EfficientDet Confusion Matrix 평가 시작")
    
    from effdet import EfficientDet, DetBenchPredict
    
    checkpoint_path = RESULT_DIR / 'efficientdet_best.pth'
    if not checkpoint_path.exists():
        print("모델 가중치 파일이 없습니다")
        return None
    
    checkpoint = torch.load(checkpoint_path, weights_only=False)
    net = EfficientDet(config, pretrained_backbone=False)
    net.load_state_dict(checkpoint['model_state_dict'])
    predictor = DetBenchPredict(net).to(device).eval()
    
    test_data = splits['test']
    
    y_true = []  
    y_pred = []  
    
    def calculate_iou(box1, box2):
        x1 = max(box1[0], box2[0])
        y1 = max(box1[1], box2[1])
        x2 = min(box1[2], box2[2])
        y2 = min(box1[3], box2[3])
        
        inter = max(0, x2 - x1) * max(0, y2 - y1)
        area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
        area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
        union = area1 + area2 - inter
        
        return inter / union if union > 0 else 0
    
    print("\n 테스트 데이터 예측 중")
    for item in tqdm(test_data, desc="Predicting"):
        with open(item['image'], 'rb') as f:
            img = cv2.imdecode(np.frombuffer(f.read(), np.uint8), cv2.IMREAD_COLOR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        h, w = img.shape[:2]
        
        img_resized = cv2.resize(img, (512, 512))
        img_tensor = torch.from_numpy(img_resized).permute(2, 0, 1).float() / 255.0
        img_tensor = img_tensor.unsqueeze(0).to(device)
        
        with torch.no_grad():
            pred = predictor(img_tensor)
        
        gt_box = item['bbox']
        gt_label = item['label']
        y_true.append(gt_label)
        
        # 예측 결과 처리
        best_pred_label = -1 
        best_iou = 0
        confidence_threshold = 0.3 
        
        if len(pred) > 0 and pred[0].shape[0] > 0:
            pred_np = pred[0].cpu().numpy()
            
            for box in pred_np:
                if box[4] < confidence_threshold:
                    continue
                
                pred_box = [
                    box[0] * w / 512, 
                    box[1] * h / 512,
                    box[2] * w / 512, 
                    box[3] * h / 512
                ]
                pred_label = int(box[5]) if len(box) > 5 else 0
                
                iou = calculate_iou(gt_box, pred_box)
                
                if iou > best_iou and iou >= 0.5:
                    best_iou = iou
                    best_pred_label = pred_label
        
        if best_pred_label == -1:
            if len(pred) > 0 and pred[0].shape[0] > 0:
                pred_np = pred[0].cpu().numpy()
                valid_preds = pred_np[pred_np[:, 4] >= confidence_threshold]
                if len(valid_preds) > 0:
                    best_idx = np.argmax(valid_preds[:, 4])
                    best_pred_label = int(valid_preds[best_idx, 5])
                else:
                    if len(pred_np) > 0:
                        best_idx = np.argmax(pred_np[:, 4])
                        best_pred_label = int(pred_np[best_idx, 5])
        
        if best_pred_label == -1:
            best_pred_label = len(classes)
        
        y_pred.append(best_pred_label)
    
    print("\n Confusion Matrix 생성 중")
    
    extended_classes = classes + ['No Detection']
    
    cm = confusion_matrix(y_true, y_pred, labels=list(range(len(extended_classes))))
    
    plt.figure(figsize=(max(12, len(extended_classes) * 0.8), max(10, len(extended_classes) * 0.7)))
    
    # 정규화된 Confusion Matrix
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    cm_normalized = np.nan_to_num(cm_normalized)
    
    sns.heatmap(cm_normalized, annot=True, fmt='.2f', cmap='Blues', 
                xticklabels=extended_classes, yticklabels=extended_classes,
                cbar_kws={'label': 'Normalized Count'})
    
    plt.title('EfficientDet Confusion Matrix (Normalized)', fontsize=14, fontweight='bold')
    plt.ylabel('True Label', fontsize=12, fontweight='bold')
    plt.xlabel('Predicted Label', fontsize=12, fontweight='bold')
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()
    
    # 저장
    cm_path = RESULT_DIR / 'efficientdet_confusion_matrix_normalized.png'
    plt.savefig(cm_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f" 정규화된 Confusion Matrix 저장: {cm_path}")
    

    plt.figure(figsize=(max(12, len(extended_classes) * 0.8), max(10, len(extended_classes) * 0.7)))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=extended_classes, yticklabels=extended_classes,
                cbar_kws={'label': 'Count'})
    
    plt.title('EfficientDet Confusion Matrix (Count)', fontsize=14, fontweight='bold')
    plt.ylabel('True Label', fontsize=12, fontweight='bold')
    plt.xlabel('Predicted Label', fontsize=12, fontweight='bold')
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()
    
    cm_count_path = RESULT_DIR / 'efficientdet_confusion_matrix_count.png'
    plt.savefig(cm_count_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f" 개수 Confusion Matrix 저장: {cm_count_path}")
    
    print("\n Classification Report:")
    print("="*70)
    # No Detection 제외한 실제 클래스만으로 리포트 생성
    valid_indices = [i for i, label in enumerate(y_pred) if label < len(classes)]
    if valid_indices:
        y_true_valid = [y_true[i] for i in valid_indices]
        y_pred_valid = [y_pred[i] for i in valid_indices]
        
        # 실제로 예측된 클래스만 추출
        unique_labels = sorted(list(set(y_true_valid + y_pred_valid)))
        target_names_subset = [classes[i] for i in unique_labels if i < len(classes)]
        
        report = classification_report(y_true_valid, y_pred_valid, 
                                       labels=unique_labels,
                                       target_names=target_names_subset, 
                                       zero_division=0)
        print(report)
        
        # Classification Report 저장
        report_path = RESULT_DIR / 'efficientdet_classification_report.txt'
        with open(report_path, 'w', encoding='utf-8') as f:
            f.write(report)
        print(f"\n Classification Report 저장: {report_path}")
    
    # 클래스별 정확도 계산
    class_accuracies = {}
    for i, class_name in enumerate(classes):
        if cm[i].sum() > 0:
            accuracy = cm[i, i] / cm[i].sum()
            class_accuracies[class_name] = accuracy
    
    # 클래스별 정확도 시각화
    if class_accuracies:
        plt.figure(figsize=(12, 6))
        sorted_classes = sorted(class_accuracies.items(), key=lambda x: x[1], reverse=True)
        class_names = [x[0] for x in sorted_classes]
        accuracies = [x[1] for x in sorted_classes]
        
        bars = plt.bar(range(len(class_names)), accuracies, color='skyblue', edgecolor='navy')
        plt.xlabel('Class', fontsize=12, fontweight='bold')
        plt.ylabel('Accuracy', fontsize=12, fontweight='bold')
        plt.title('EfficientDet: Per-Class Accuracy', fontsize=14, fontweight='bold')
        plt.xticks(range(len(class_names)), class_names, rotation=45, ha='right')
        plt.ylim(0, 1.1)
        plt.grid(axis='y', alpha=0.3)
        
        # 값 표시
        for bar in bars:
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2., height,
                    f'{height:.2f}',
                    ha='center', va='bottom', fontsize=9)
        
        plt.tight_layout()
        acc_path = RESULT_DIR / 'efficientdet_per_class_accuracy.png'
        plt.savefig(acc_path, dpi=300, bbox_inches='tight')
        plt.close()
        print(f" 클래스별 정확도 그래프 저장: {acc_path}")
    
    # 전체 정확도 계산
    total_correct = np.trace(cm[:len(classes), :len(classes)])  # No Detection 제외
    total_samples = cm[:len(classes), :].sum()
    overall_accuracy = total_correct / total_samples if total_samples > 0 else 0
    
    print(f"\n 전체 정확도: {overall_accuracy:.3f}")
    print(f" 총 테스트 샘플: {len(y_true)}개")
    print(f" 정확히 분류된 샘플: {total_correct}개")
    
    # Confusion Matrix 데이터 저장
    cm_data = {
        'confusion_matrix': cm.tolist(),
        'classes': extended_classes,
        'overall_accuracy': float(overall_accuracy),
        'class_accuracies': {k: float(v) for k, v in class_accuracies.items()},
        'total_samples': int(total_samples),
        'correct_predictions': int(total_correct)
    }
    
    cm_json_path = RESULT_DIR / 'efficientdet_confusion_matrix.json'
    import json
    with open(cm_json_path, 'w', encoding='utf-8') as f:
        json.dump(cm_data, f, indent=2, ensure_ascii=False)
    print(f" Confusion Matrix 데이터 저장: {cm_json_path}")
    
    return cm, class_accuracies, overall_accuracy

In [11]:
def test_efficientdet(config, splits, classes):
    print("\n EfficientDet 테스트셋 평가 시작")
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    

    if COCO_AVAILABLE and len(splits['test']) > 0:
        gt_anno_file = DATASET_EFFDET / 'coco_test.json'
        effdet_test_metrics = evaluate_efficientdet_coco(config, splits, classes, device, gt_anno_file)
    else:
        effdet_test_metrics = evaluate_efficientdet_simple(config, splits, classes, device)

    cm, class_accuracies, overall_accuracy = evaluate_efficientdet_with_confusion_matrix(
        config, splits, classes, device
    )
    
    effdet_test_metrics['overall_accuracy'] = overall_accuracy
    effdet_test_metrics['class_accuracies'] = class_accuracies
    
    effdet_test_metrics.setdefault('mAP50', 0.0)
    effdet_test_metrics.setdefault('mAP50_95', 0.0)
    effdet_test_metrics.setdefault('precision', 0.0)
    effdet_test_metrics.setdefault('recall', 0.0)
    
    print(f"\nEfficientDet 테스트 완료")
    print(f" mAP@0.5: {effdet_test_metrics['mAP50']:.3f}")
    print(f" mAP@0.5:0.95: {effdet_test_metrics['mAP50_95']:.3f}")
    print(f" Precision: {effdet_test_metrics['precision']:.3f}")
    print(f" Recall: {effdet_test_metrics['recall']:.3f}")
    print(f" Overall Accuracy: {overall_accuracy:.3f}")
    
    return effdet_test_metrics

In [12]:
def evaluate_efficientdet_coco(config, splits, classes, device, gt_anno_file):
    print("\nEfficientDet COCO 평가 시작")

    checkpoint_path = RESULT_DIR / 'efficientdet_best.pth'
    if not checkpoint_path.exists():
        print("모델 가중치 파일이 없습니다:", checkpoint_path)
        return {'mAP50': 0.0, 'mAP50_95': 0.0, 'precision': 0.0, 'recall': 0.0}

    checkpoint = torch.load(checkpoint_path, weights_only=False)
    net = EfficientDet(config, pretrained_backbone=False)
    net.load_state_dict(checkpoint['model_state_dict'])

    bench = DetBenchPredict(net)
    bench.eval()
    bench.to(device)

    coco_gt = COCO(str(gt_anno_file))

    results = []
    test_data = splits['test']

    vis_dir = RESULT_DIR / 'efficientdet_test_predictions'
    vis_dir.mkdir(exist_ok=True)

    for img_id, item in enumerate(tqdm(test_data, desc="Predicting on Test Set")):
        img = cv2.imread(item['image'])
        if img is None:
            continue
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        h, w = img_rgb.shape[:2]

        img_resized = cv2.resize(img_rgb, (512, 512))
        img_tensor = torch.from_numpy(img_resized).permute(2, 0, 1).float() / 255.0
        img_tensor = img_tensor.unsqueeze(0).to(device)

        with torch.no_grad():
            detections = bench(img_tensor)

        if detections is None or len(detections.shape) != 3:
            continue

        det = detections[0].cpu().numpy()
        for i in range(det.shape[0]):
            if det.shape[1] < 6:
                continue
            score = float(det[i, 4])
            class_id = int(det[i, 5])
            if score < 0.001:
                continue

            x1, y1, x2, y2 = det[i, :4]
            x1 = float(x1 * w / 512)
            y1 = float(y1 * h / 512)
            x2 = float(x2 * w / 512)
            y2 = float(y2 * h / 512)
            
            if x2 <= x1 or y2 <= y1 or class_id < 0 or class_id >= len(classes):
                continue

            results.append({
                'image_id': int(img_id),
                'category_id': int(class_id),
                'bbox': [x1, y1, x2 - x1, y2 - y1],
                'score': float(score)
            })

    if not results:
        print("예측 결과 없음")
        return {'mAP50': 0.0, 'mAP50_95': 0.0, 'precision': 0.0, 'recall': 0.0}

    pred_file = RESULT_DIR / 'coco_test_predictions.json'
    with open(pred_file, 'w') as f:
        json.dump(results, f)

    try:
        coco_dt = coco_gt.loadRes(str(pred_file))
        coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')
        coco_eval.evaluate()
        coco_eval.accumulate()
        coco_eval.summarize()

        metrics = {
            'mAP50_95': float(coco_eval.stats[0]),
            'mAP50': float(coco_eval.stats[1]),
            'precision': float(coco_eval.stats[0]),
            'recall': float(coco_eval.stats[8])
        }

    except Exception as e:
        print(f"COCO 평가 중 오류: {e}")
        return {'mAP50': 0.0, 'mAP50_95': 0.0, 'precision': 0.0, 'recall': 0.0}
    
        # EfficientDet metrics 저장
    effdet_metrics_path = RESULT_DIR / 'efficientdet_metrics.json'
    with open(effdet_metrics_path, 'w', encoding='utf-8') as f:
        json.dump({
            'summary': metrics
        }, f, indent=4, ensure_ascii=False)
    print(f"EfficientDet metrics 저장: {effdet_metrics_path}")

    return metrics

In [13]:
def evaluate_efficientdet_simple(config, splits, classes, device):
    print("\n EfficientDet Simple 평가 중")
    
    checkpoint_path = RESULT_DIR / 'efficientdet_best.pth'
    if not checkpoint_path.exists():
        print("모델 가중치 파일이 없습니다")
        return {'mAP50': 0.0, 'mAP50_95': 0.0, 'precision': 0.0, 'recall': 0.0}
    
    checkpoint = torch.load(checkpoint_path, weights_only=False)
    net = EfficientDet(config, pretrained_backbone=False)
    net.load_state_dict(checkpoint['model_state_dict'])
    predictor = DetBenchPredict(net).to(device).eval()
    
    test_data = splits['test']
    
    def calculate_iou(box1, box2):
        x1 = max(box1[0], box2[0])
        y1 = max(box1[1], box2[1])
        x2 = min(box1[2], box2[2])
        y2 = min(box1[3], box2[3])
        
        inter = max(0, x2 - x1) * max(0, y2 - y1)
        area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
        area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
        union = area1 + area2 - inter
        
        return inter / union if union > 0 else 0
    
    correct_50 = 0
    all_ious = []
    
    for item in tqdm(test_data, desc="Evaluating"):
        with open(item['image'], 'rb') as f:
            img = cv2.imdecode(np.frombuffer(f.read(), np.uint8), cv2.IMREAD_COLOR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        h, w = img.shape[:2]
        
        img_resized = cv2.resize(img, (512, 512))
        img_tensor = torch.from_numpy(img_resized).permute(2, 0, 1).float() / 255.0
        img_tensor = img_tensor.unsqueeze(0).to(device)
        
        with torch.no_grad():
            pred = predictor(img_tensor)
        
        gt_box = item['bbox']
        gt_label = item['label']
        
        best_iou = 0
        
        if len(pred) > 0 and pred[0].shape[0] > 0:
            pred_np = pred[0].cpu().numpy()
            for box in pred_np:
                if box[4] < 0.1:
                    continue
                
                pred_box = [
                    box[0] * w / 512, box[1] * h / 512,
                    box[2] * w / 512, box[3] * h / 512
                ]
                pred_label = int(box[5]) if len(box) > 5 else 0
                
                if pred_label == gt_label:
                    iou = calculate_iou(gt_box, pred_box)
                    best_iou = max(best_iou, iou)
        
        all_ious.append(best_iou)
        if best_iou >= 0.5:
            correct_50 += 1
    
    total = len(test_data)
    mAP_50 = correct_50 / total if total > 0 else 0
    
    mAP_50_95_sum = 0
    for thresh in [0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95]:
        correct = sum([1 for iou in all_ious if iou >= thresh])
        mAP_50_95_sum += (correct / total if total > 0 else 0)
    mAP_50_95 = mAP_50_95_sum / 10
    
    metrics = {
        'mAP50': float(mAP_50),
        'mAP50_95': float(mAP_50_95),
        'precision': float(mAP_50),
        'recall': float(mAP_50)
    }

        # EfficientDet metrics 저장
    effdet_metrics_path = RESULT_DIR / 'efficientdet_metrics.json'
    with open(effdet_metrics_path, 'w', encoding='utf-8') as f:
        json.dump({
            'summary': metrics
        }, f, indent=4, ensure_ascii=False)
    print(f"EfficientDet metrics 저장: {effdet_metrics_path}")
    
    return metrics

In [14]:
def visualize_comparison(yolo_metrics, effdet_metrics, title_suffix=""):
    metrics_names = ['mAP@0.5', 'mAP@0.5:0.95', 'Precision', 'Recall']
    yolo_values = [
        yolo_metrics['mAP50'],
        yolo_metrics['mAP50_95'],
        yolo_metrics['precision'],
        yolo_metrics['recall']
    ]
    effdet_values = [
        effdet_metrics['mAP50'],
        effdet_metrics['mAP50_95'],
        effdet_metrics['precision'],
        effdet_metrics['recall']
    ]
    
    x = np.arange(len(metrics_names))
    width = 0.35
    
    fig, ax = plt.subplots(figsize=(12, 6))
    bars1 = ax.bar(x - width/2, yolo_values, width, label='YOLOv5', color='#FF6B6B')
    bars2 = ax.bar(x + width/2, effdet_values, width, label='EfficientDet', color='#4ECDC4')
    
    ax.set_xlabel('Metrics', fontsize=12, fontweight='bold')
    ax.set_ylabel('Score', fontsize=12, fontweight='bold')
    ax.set_title(f'YOLOv5 vs EfficientDet Performance Comparison{title_suffix}', fontsize=14, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(metrics_names)
    ax.legend(fontsize=11)
    ax.set_ylim(0, 1.1)
    ax.grid(axis='y', alpha=0.3)
    
    for bars in [bars1, bars2]:
        for bar in bars:
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height,
                   f'{height:.3f}',
                   ha='center', va='bottom', fontsize=9)
    
    plt.tight_layout()
    filename = 'performance_comparison_test.png' if title_suffix else 'performance_comparison.png'
    plt.savefig(RESULT_DIR / filename, dpi=300, bbox_inches='tight')
    print(f"\n 비교 그래프 저장: {RESULT_DIR / filename}")
    plt.close()

In [15]:
def print_results_table(yolo_metrics, effdet_metrics, title="성능 비교"):
    print("\n" + "="*70)
    print(f"{title:^70}")
    print("="*70)
    print(f"\n{'Metric':<20} {'YOLOv5':<15} {'EfficientDet':<15} {'Difference':<15}")
    print("-"*70)
    print(f"{'mAP@0.5':<20} {yolo_metrics['mAP50']:<15.3f} {effdet_metrics['mAP50']:<15.3f} {yolo_metrics['mAP50']-effdet_metrics['mAP50']:+.3f}")
    print(f"{'mAP@0.5:0.95':<20} {yolo_metrics['mAP50_95']:<15.3f} {effdet_metrics['mAP50_95']:<15.3f} {yolo_metrics['mAP50_95']-effdet_metrics['mAP50_95']:+.3f}")
    print(f"{'Precision':<20} {yolo_metrics['precision']:<15.3f} {effdet_metrics['precision']:<15.3f} {yolo_metrics['precision']-effdet_metrics['precision']:+.3f}")
    print(f"{'Recall':<20} {yolo_metrics['recall']:<15.3f} {effdet_metrics['recall']:<15.3f} {yolo_metrics['recall']-effdet_metrics['recall']:+.3f}")
    
    if yolo_metrics['mAP50'] > effdet_metrics['mAP50']:
        winner = "YOLOv5"
        diff = yolo_metrics['mAP50'] - effdet_metrics['mAP50']
    elif effdet_metrics['mAP50'] > yolo_metrics['mAP50']:
        winner = "EfficientDet"
        diff = effdet_metrics['mAP50'] - yolo_metrics['mAP50']
    else:
        winner = "동점"
        diff = 0
    
    print(f"\n{'='*70}")
    if winner != "동점":
        print(f"{winner}가 {diff:.3f}만큼 더 높은 mAP@0.5를 달성했습니다!")
    else:
        print(f"두 모델이 동일한 성능을 보였습니다!")
    print(f"{'='*70}")
    
    return winner

In [16]:
def compare_models_final():
    yolo_metrics_path = RESULT_DIR / "yolo_metrics.json"
    effdet_metrics_path = RESULT_DIR / "efficientdet_metrics.json"
    
    if not yolo_metrics_path.exists() or not effdet_metrics_path.exists():
        print("metrics 파일이 없습니다. 평가가 완료되지 않았습니다.")
        return

    with open(yolo_metrics_path, "r", encoding="utf-8") as f:
        yolo_data = json.load(f)
    with open(effdet_metrics_path, "r", encoding="utf-8") as f:
        effdet_data = json.load(f)

    yolo_summary = yolo_data.get("summary", {})
    effdet_summary = effdet_data.get("summary", {})

    metrics_names = ["mAP@0.5", "mAP@0.5:0.95", "Precision", "Recall"]
    metric_keys = ["mAP50", "mAP50_95", "precision", "recall"]

    yolo_scores = [yolo_summary.get(k, 0) for k in metric_keys]
    effdet_scores = [effdet_summary.get(k, 0) for k in metric_keys]

    x = np.arange(len(metrics_names))
    width = 0.35

    plt.figure(figsize=(10, 6))
    plt.bar(x - width/2, yolo_scores, width, label="YOLOv5", color='red', alpha=0.8)
    plt.bar(x + width/2, effdet_scores, width, label="EfficientDet", color='blue', alpha=0.8)

    plt.xticks(x, metrics_names, fontsize=11)
    plt.ylabel("Score", fontsize=12, fontweight='bold')
    plt.ylim(0, 1)
    plt.title("YOLOv5 vs EfficientDet 최종 성능 비교", fontsize=14, fontweight='bold')
    plt.legend(fontsize=11)
    plt.grid(axis='y', alpha=0.3)
    
    # 값 표시
    for i, (y_score, e_score) in enumerate(zip(yolo_scores, effdet_scores)):
        if y_score > 0:
            plt.text(i - width/2, y_score, f'{y_score:.3f}', ha='center', va='bottom', fontsize=9)
        if e_score > 0:
            plt.text(i + width/2, e_score, f'{e_score:.3f}', ha='center', va='bottom', fontsize=9)
    
    plt.tight_layout()

    save_path = RESULT_DIR / "final_comparison_graph.png"
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()

    print(f"\n 최종 성능 비교 그래프 저장 완료 → {save_path}")

In [17]:
def main():
    print("YOLOv5 vs EfficientDet 성능 비교 (실제 테스트 데이터)")
    
    # 1. 데이터 전처리
    print("\n 1단계 데이터 전처리")
    print(f"   원천데이터 경로: {JSON_DIR}")
    
    splits, classes = preprocess_data()
    if not classes:
        print("데이터셋 없음")
        return
    
    if len(splits['test']) == 0:
        print("\n 오류: 테스트 데이터가 없습니다!")
        print(f"   다음 경로를 확인해주세요:")
        return
    
    print(f"\n클래스 목록 ({len(classes)}개):")
    for i, cls in enumerate(classes):
        print(f"  {i}: {cls}")
    
    # 2. YOLOv5 학습
    print("2단계 YOLOv5 학습")
    prepare_yolo_dataset(splits, classes)
    yolo_model = train_yolo(DATASET_YOLO / 'data.yaml', epochs=5)

    # 3. EfficientDet 학습
    print("3단계 EfficientDet 학습")
    effdet_model, effdet_config = train_efficientdet(splits, classes, epochs=30)
    
    # 4. 테스트셋 평가
    print("4단계 실제 테스트 데이터로 최종 평가")
    print(f"   테스트 이미지 수: {len(splits['test'])}개")
    
    # YOLOv5 테스트
    yolo_test_metrics = test_yolo(yolo_model, DATASET_YOLO / 'data.yaml')
    
    # EfficientDet 테스트
    effdet_test_metrics = test_efficientdet(effdet_config, splits, classes)
    
    # 5. 결과 비교 및 시각화
    print("【5단계】 결과 비교 및 시각화")
    
    winner = print_results_table(yolo_test_metrics, effdet_test_metrics, "실제 테스트 데이터 최종 성능 비교")
    visualize_comparison(yolo_test_metrics, effdet_test_metrics, " (Real Test Data)")
    
    # 최종 비교 그래프
    compare_models_final()
    
    # 6. 결과 저장
    results_summary = {
        'dataset_info': {
            'train_source': 'data/raw (80%)',
            'val_source': 'data/raw (20%)',
            'test_source': 'data/test_data (별도)',
            'num_train': len(splits['train']),
            'num_val': len(splits['val']),
            'num_test': len(splits['test']),
            'classes': classes
        },
        'yolo_test': yolo_test_metrics,
        'efficientdet_test': effdet_test_metrics,
        'winner': winner
    }
    
    with open(RESULT_DIR / 'final_test_results.json', 'w', encoding='utf-8') as f:
        json.dump(results_summary, f, indent=2, ensure_ascii=False)
    
    # 7. 간단한 요약 그래프
    labels = ['mAP@0.5', 'mAP@0.5:0.95']
    yolo_scores = [yolo_test_metrics['mAP50'], yolo_test_metrics['mAP50_95']]
    effdet_scores = [effdet_test_metrics['mAP50'], effdet_test_metrics['mAP50_95']]

    x = np.arange(len(labels))
    width = 0.35

    plt.figure(figsize=(8, 5))
    plt.bar(x - width/2, yolo_scores, width, label='YOLOv5', color='red', alpha=0.8)
    plt.bar(x + width/2, effdet_scores, width, label='EfficientDet', color='blue', alpha=0.8)

    plt.ylabel('Score', fontsize=12, fontweight='bold')
    plt.title('Real Test Data: YOLOv5 vs EfficientDet', fontsize=14, fontweight='bold')
    plt.xticks(x, labels, fontsize=11)
    plt.legend(fontsize=11)
    plt.ylim(0, 1.1)
    plt.grid(True, axis='y', linestyle='--', alpha=0.3)
    
    for i, (y_score, e_score) in enumerate(zip(yolo_scores, effdet_scores)):
        plt.text(i - width/2, y_score, f'{y_score:.3f}', ha='center', va='bottom', fontsize=9)
        plt.text(i + width/2, e_score, f'{e_score:.3f}', ha='center', va='bottom', fontsize=9)
    
    plt.tight_layout()
    plt.savefig(RESULT_DIR / 'test_summary_graph.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    # 최종 출력
    print("모든 작업 완료!")
    print(f"\n 결과 저장 위치: {RESULT_DIR.resolve()}")
    print(f"\n 생성된 파일:")
    print(f"  YOLOv5 테스트: yolov5_test/")
    print(f"  EfficientDet 테스트 예측: efficientdet_test_predictions/")
    print(f"  성능 비교 그래프: performance_comparison_test.png")
    print(f"  최종 비교 그래프: final_comparison_graph.png")
    print(f"  요약 그래프: test_summary_graph.png")
    print(f"  결과 JSON: final_test_results.json")
    print(f"  YOLO metrics: yolo_metrics.json")
    print(f"  EfficientDet metrics: efficientdet_metrics.json")
    
    print(f"\n 데이터 구성:")
    print(f"  - 학습: {len(splits['train'])}개 (원천데이터 80%)")
    print(f"  - 검증: {len(splits['val'])}개 (원천데이터 20%)")
    print(f"  - 테스트: {len(splits['test'])}개 (별도 실제 데이터)")
    
    print(f"\n 최종 승자: {winner}")

In [18]:
if __name__ == "__main__":
    main()

YOLOv5 vs EfficientDet 성능 비교 (실제 테스트 데이터)

 1단계 데이터 전처리
   원천데이터 경로: c:\miniproject\data정리\data\raw\json_labels


Loading test: 100%|██████████| 9/9 [00:00<00:00, 2246.01it/s]


Dataset: train=72, val=9, test=9

클래스 목록 (9개):
  0: 배_상
  1: 감_특
  2: 감_상
  3: 배_특
  4: 사과_상
  5: 사과_특
  6: 사과_보통
  7: 감_보통
  8: 배_보통
2단계 YOLOv5 학습


YOLO test: 100%|██████████| 9/9 [00:00<00:00, 39.56it/s]


YOLO 데이터셋 준비 완료: c:\miniproject\data정리\processed\preprocessed_data\yolov5

 YOLOv5 학습 시작
New https://pypi.org/project/ultralytics/8.3.227 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.225  Python-3.9.25 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=c:\miniproject\data\processed\preprocessed_data\yolov5\data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4,

...

Epoch 30: Train=0.1154, Val=0.3370
EfficientDet 학습 완료 (Best Loss: 0.3368)
 EfficientDet 학습 곡선 저장됨: c:\miniproject\data정리\processed\results_comparison\efficientdet_loss_curve.png
4단계 실제 테스트 데이터로 최종 평가
   테스트 이미지 수: 9개

 YOLOv5 테스트셋 평가 시작
Ultralytics 8.3.225  Python-3.9.25 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
YOLOv5s summary (fused): 84 layers, 9,115,019 parameters, 0 gradients, 23.8 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 17.94.9 MB/s, size: 202.6 KB)
val: Scanning C:\miniproject\data정리\processed\preprocessed_data\yolov5\labels\test... 9 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 9/9 682.1it/s 0.0s
val: New cache created: C:\miniproject\data\processed\preprocessed_data\yolov5\labels\test.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 1/1 0.3it/s 3.5s
                   all          9          9      0.237      0.832      0.444      0.375
                   _      

Predicting on Test Set: 100%|██████████| 9/9 [00:00<00:00, 12.42it/s]


Loading and preparing results...
DONE (t=0.01s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.02s).
Accumulating evaluation results...
DONE (t=0.03s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.764
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.786
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.786
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.764
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.814
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.814
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.814
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=

Predicting: 100%|██████████| 9/9 [00:00<00:00, 14.65it/s]
C:\Users\smart\AppData\Local\Temp\ipykernel_3300\3296030768.py:107: RuntimeWarning: invalid value encountered in divide
  cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]



 Confusion Matrix 생성 중
 정규화된 Confusion Matrix 저장: c:\miniproject\data정리\processed\results_comparison\efficientdet_confusion_matrix_normalized.png
 개수 Confusion Matrix 저장: c:\miniproject\data정리\processed\results_comparison\efficientdet_confusion_matrix_count.png

 Classification Report:
              precision    recall  f1-score   support

         배_상       0.00      0.00      0.00         1
         감_상       0.00      0.00      0.00         1
         배_특       0.50      1.00      0.67         1
        사과_상       1.00      1.00      1.00         2
        사과_특       0.50      1.00      0.67         1
       사과_보통       1.00      0.50      0.67         2
        감_보통       0.00      0.00      0.00         1

    accuracy                           0.56         9
   macro avg       0.43      0.50      0.43         9
weighted avg       0.56      0.56      0.52         9


 Classification Report 저장: c:\miniproject\data정리\processed\results_comparison\efficientdet_classification_report.t